In [2]:
import pymysql
import pandas as pd

# 공통
%matplotlib inline
import numpy as np
import statsmodels.api as sm

# 예측 1
import matplotlib.pyplot as plt
import datetime
from dateutil.relativedelta import relativedelta
import seaborn as sns
from statsmodels.tsa.stattools import acf
from statsmodels.tsa.stattools import pacf
from statsmodels.tsa.seasonal import seasonal_decompose

# 예측 2
from scipy.stats import norm
from tqdm import tqdm
from sklearn.metrics import mean_squared_error

In [5]:
con = pymysql.connect(host = '127.0.0.1', user = 'root', password = 'kyjin0413!',
                       db = 'lpointsample', charset = 'utf8')
cur = con.cursor(pymysql.cursors.DictCursor)
cur.execute(
    "select de_dt as 날짜, de_hr as 시간대, sum(buy_am) as 금액 ,count(*) as 이용횟수 from 엘페이이용_view group by de_dt, de_hr order by de_dt, de_hr")
result=cur.fetchall()
con.commit()
con.close()
df= pd.DataFrame(result)
# print(df.head(10))

df.columns = ['date', 'time', 'price', 'count_days']
# df = df.drop(['date'], axis = 1)
df.tail(31)

,date,time,price,count_days
8602,2021-12-30,17,2364835,73
8603,2021-12-30,18,2202192,66
8604,2021-12-30,19,2278985,65
8605,2021-12-30,20,1505973,48
8606,2021-12-30,21,1925676,39
8607,2021-12-30,22,2026283,36
8608,2021-12-30,23,1276070,25
8609,2021-12-31,0,815660,11
8610,2021-12-31,1,646027,13
8611,2021-12-31,2,280158,7


In [18]:
df.dtypes

date          object
time           int64
price         object
count_days     int64
dtype: object

In [17]:
df = df.astype({'time':'int64'}) 

In [14]:
pd.concat(['date', 'time'], axis = 0)

TypeError: cannot concatenate object of type '<class 'str'>'; only Series and DataFrame objs are valid

In [50]:
# # 인덱스 지정(?)

# # strptime은 날짜와 시간 형식의 문자열을 datetime으로 변환해 줌
# # strftime은 날짜와 시간(datetime)을 문자열로 출력
# start = datetime.datetime.strptime("2021-01-01", "%Y-%m-%d")

# # 기준 날짜에 대한 연산이 필요할 때 dateutil.relativedelta모듈의 relativedelta함수 이용
# df_list = [start + relativedelta(days = x) for x in range(0, len(df)) ]

# df['index'] = df_list

# # set_index는 데이터프레임을 먼저 불러온 후, 특정 열을 인덱스로 설정할 때 사용
# df.set_index( ['index'], inplace=True) 

# # index 출력 X
# df.index.name=None
# df.head()

,date,time,price,count_days
2021-01-01,2021-01-01,0,2683380,72
2021-01-02,2021-01-01,1,499360,11
2021-01-03,2021-01-01,2,64176,10
2021-01-04,2021-01-01,3,885833,11
2021-01-05,2021-01-01,5,37630,1


In [56]:
df.dtypes

date          object
time           int64
price         object
count_days     int64
dtype: object

In [57]:
# price가 object임
df = df.astype({'price':'int64'})

In [58]:
df.dtypes

date          object
time           int64
price          int64
count_days     int64
dtype: object

In [59]:
# coef값은 0에서 가장 떨어졌을때 가장 이상적
# p 값은 0과 가장 가까울때가 가장 좋습니다

mod = sm.tsa.statespace.SARIMAX(
    df['price'],
    order = (1, 1, 1),
    seasonal_order = (0, 1, 1, 24),
    # AR 매개 변수를 변환하여 모델의 자동 회귀 구성 요소에서 정상 성을 강제 적용할지 여부입니다. 기본값은 True입니다.
    # AR항이 stationary를 만족하게끔 강제하는 것
    enforce_stationarity = False,
    # 모델의 이동 평균 구성 요소에서 반전 성을 강제하기 위해 MA 매개 변수를 변환할지 여부입니다. 기본값은 True입니다.
    # MA항이 starionary를 만족하게끔 강제하는 것
    enforce_invertibility = False
)

results = mod.fit()

# print(results.summary())
print(results.summary().tables[1])

                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.2283      0.012     18.408      0.000       0.204       0.253
ma.L1         -0.9696      0.005   -201.713      0.000      -0.979      -0.960
ma.S.L24      -0.8327      0.008   -101.025      0.000      -0.849      -0.817
sigma2       1.57e+13   1.08e-16   1.46e+29      0.000    1.57e+13    1.57e+13


In [60]:
# Get forecast some steps ahead in future 미래 예측
pred_uc = results.get_forecast( steps = 31 )

In [61]:
forecast_mean = pred_uc.predicted_mean
forecast_mean

8633    1.906872e+06
8634    1.821569e+06
8635    1.290710e+06
8636    1.076140e+06
8637    8.930985e+05
8638    6.809518e+05
8639    6.496479e+05
8640    6.547626e+05
8641    1.001119e+06
8642    1.089976e+06
8643    1.395273e+06
8644    3.673076e+06
8645    1.882549e+06
8646    3.988142e+06
8647    4.460141e+06
8648    4.583753e+06
8649    3.198083e+06
8650    3.710727e+06
8651    6.491237e+06
8652    3.608868e+06
8653    3.470332e+06
8654    2.884417e+06
8655    2.927550e+06
8656    2.216562e+06
8657    2.100384e+06
8658    1.896725e+06
8659    1.338849e+06
8660    1.118112e+06
8661    9.336630e+05
8662    7.211951e+05
8663    6.898178e+05
Name: predicted_mean, dtype: float64

In [62]:
forecast_mean.to_csv('lpoint_week.csv', header=['forecast'], index=False)